In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../working"))
print(os.listdir("../input/cats-and-dogs-small/cats_and_dogs_small/cats_and_dogs_small/train"))
print(os.listdir("../input/dogs-vs-cats"))
# Any results you write to the current directory are saved as output.

In [ ]:
# Marking the necessary directories 
train_dir = "../input/cats-and-dogs-small/cats_and_dogs_small/cats_and_dogs_small/train"
train_cats_dir = os.path.join(train_dir, 'cats')

validation_dir = "../input/cats-and-dogs-small/cats_and_dogs_small/cats_and_dogs_small/validation"
test_dir = "../input/cats-and-dogs-small/cats_and_dogs_small/cats_and_dogs_small/test"

In [ ]:
# model_1
from keras.models import Sequential
from keras import layers

model = Sequential()

model.add(layers.Conv2D(32, (3,3), activation='relu',
                        input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64, (3,3), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(128, (3,3), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(128, (3,3), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.summary()

In [ ]:
# configuring model_1 for training
from keras import optimizers

model.compile(loss='binary_crossentropy',
             optimizer=optimizers.RMSprop(lr=1e-4),
             metrics=['acc'])

In [ ]:
"""
Data preprocessing : using ImageDataGenerator to turn image files on disk into 
batches of preprocessed tensors.
Note that the generator yields these batches indefinitely:
it loops endlessly over the images in the target folder
"""
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary')

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary')


In [ ]:
# checking the shape of train_generator
for data_batch, label_batch in train_generator:
    print('data batch shape : ', data_batch.shape)
    print('label batch shape : ', label_batch.shape)
    break

In [ ]:
#  Fitting the model_1 using a batch generator
history = model.fit_generator(train_generator,
                              steps_per_epoch=100,
                              epochs=30,
                              validation_data=validation_generator,
                              validation_steps=50)

In [ ]:
# saving model_1
model.save('cats_and_dogs_small_1.h5')

In [ ]:
# Displaying curves of loss and accuracy during training of model_1
import matplotlib.pyplot as plt

acc =  history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')

plt.legend()
plt.show()

plt.plot(epochs, loss, 'ro', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')

plt.legend()
plt.show()

In [ ]:
"""
validation accuracy stalls at ~72%, now using 'data augmentation'
Setting up a data augmentation configuration via ImageDataGenerator
"""
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rotation_range=40,
                             width_shift_range=0.2,
                             height_shift_range=0.2,
                             shear_range=0.2,
                             zoom_range=0.2,
                             horizontal_flip=True,
                             fill_mode='nearest')


In [ ]:
# checking ImageDataGenerator
from keras.preprocessing import image
import matplotlib.pyplot as plt

fname = [os.path.join(train_cats_dir, fname) for 
         fname in os.listdir(train_cats_dir)]

img_path = fname[79]

img = image.load_img(img_path, target_size=(150, 150))

x = image.img_to_array(img)
x = x.reshape((1,) + x.shape)
print(x.shape)

i = 0
for batch in datagen.flow(x, batch_size=1):
    plt.figure(i)
    imgplot = plt.imshow(image.array_to_img(batch[0]))
    i += 1
    if i%4 == 0:
        break
plt.show()

In [ ]:
# model_2 that includes dropout
from keras.models import Sequential
from keras import layers

model = Sequential()

model.add(layers.Conv2D(32, (3,3), activation='relu',
                        input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64, (3,3), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(128, (3,3), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(128, (3,3), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.summary()

In [ ]:
# configuring model_2 for training
from keras import optimizers

model.compile(loss='binary_crossentropy',
             optimizer=optimizers.RMSprop(lr=1e-4),
             metrics=['acc'])

In [ ]:
# Training the model_2 using data-augmentation generators
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=40,
                             width_shift_range=0.2,
                             height_shift_range=0.2,
                             shear_range=0.2,
                             zoom_range=0.2,
                             horizontal_flip=True,
                             fill_mode='nearest')

validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary')

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary')

history = model.fit_generator(train_generator,
                              steps_per_epoch=100,
                              epochs=100,
                              validation_data=validation_generator,
                              validation_steps=50)

In [ ]:
# saving model_2
model.save('cats_and_dogs_small_2.h5')

In [ ]:
# Displaying curves of loss and accuracy during training of model_2
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epoch = range(1, len(acc) + 1)

plt.plot(epoch, acc, 'bo', label='Training accuracy')
plt.plot(epoch, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')

plt.legend()
plt.show()

plt.plot(epoch, loss, 'ro', label='Training loss')
plt.plot(epoch, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')

plt.legend()
plt.show()

In [ ]:
"""
Using a pretrained convnet
There are two ways to use a pretrained network: feature extraction and fine-tuning
"""
#  Instantiating the VGG16 convolutional base
from keras.applications import VGG16

conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(150, 150, 3))

In [ ]:
# summary of VGG16
conv_base.summary()

In [ ]:
"""
FAST FEATURE EXTRACTION WITHOUT DATA AUGMENTATION
"""
#  Extracting features using the pretrained convolutional base
datagen = ImageDataGenerator(rescale=1./255)
batch_size = 20

def extract_features(directory, sample_count):
    features = np.zeros(shape=(sample_count, 4, 4, 512))
    labels = np.zeros(shape=(sample_count))
    generator = datagen.flow_from_directory(directory,
                                            target_size=(150, 150),
                                            batch_size=batch_size,
                                            class_mode='binary')
    i=0
    for inputs_batch, labels_batch in generator:
        features_batch = conv_base.predict(inputs_batch)
        features[i*batch_size:(i+1)*batch_size] = features_batch
        labels[i * batch_size : (i+1) * batch_size] = labels_batch
        i+=1
        if i*batch_size >= sample_count:
            break
    return features, labels

train_features, train_labels = extract_features(train_dir, 2000)
validation_features, validation_labels = extract_features(validation_dir, 1000)
test_features, test_labels = extract_features(test_dir, 1000)

In [ ]:
train_features = np.reshape(train_features, (2000, 4*4*512))
validation_features = np.reshape(validation_features, (1000, 4*4*512))
test_features = np.reshape(test_features, (1000, 4*4*512))

In [ ]:
# Defining and training the densely connected classifier model_3
from keras import models, layers, optimizers

model = models.Sequential()

model.add(layers.Dense(256, activation='relu', input_dim=4*4*512))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer=optimizers.RMSprop(lr=2e-5),
              loss='binary_crossentropy',
              metrics=['acc'])

history = model.fit(train_features,
                    train_labels,
                    epochs=30,
                    batch_size=20,
                    validation_data=(validation_features, validation_labels))

In [ ]:
# Displaying curves of loss and accuracy during training of model_3
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epoch = range(1, len(acc) + 1)

plt.plot(epoch, acc, 'bo', label='Training accuracy')
plt.plot(epoch, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')

plt.legend()
plt.show()

plt.plot(epoch, loss, 'ro', label='Training loss')
plt.plot(epoch, val_loss, 'r', label='Validation loss')
plt.title('Trainign and validation loss')

plt.legend()
plt.show()

In [ ]:
"""
FEATURE EXTRACTION WITH DATA AUGMENTATION
"""
#  Adding a densely connected classifier on top of the convolutional base model_4
from keras import models, layers

model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [ ]:
# summary of model_4
model.summary()

In [ ]:
# freezing the conv_base network so that weights don't update
print('This is the number of trainable weights '
'before freezing the conv base:', len(model.trainable_weights))
conv_base.trainable = False
print('This is the number of trainable weights '
'after freezing the conv base:', len(model.trainable_weights))

In [ ]:
# configuring model_4 for training
from keras import optimizers

model.compile(loss='binary_crossentropy',
             optimizer=optimizers.RMSprop(lr=1e-4),
             metrics=['acc'])

In [ ]:
#  Training the model_4 end to end with a frozen convolutional base
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=40,
                             width_shift_range=0.2,
                             height_shift_range=0.2,
                             shear_range=0.2,
                             zoom_range=0.2,
                             horizontal_flip=True,
                             fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary')

history = model.fit_generator(train_generator,
                              steps_per_epoch=100,
                              epochs=30,
                              validation_data=validation_generator,
                              validation_steps=50)

In [ ]:
# Displaying curves of loss and accuracy during training of model_4
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epoch = range(1, len(acc) + 1)

plt.plot(epoch, acc, 'bo', label='Training accuracy')
plt.plot(epoch, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')

plt.legend()
plt.show()

plt.plot(epoch, loss, 'ro', label='Training loss')
plt.plot(epoch, val_loss, 'r', label='Validation loss')
plt.title('Trainign and validation loss')

plt.legend()
plt.show()

In [ ]:
test_generator = test_datagen.flow_from_directory(test_dir,
                                                  target_size=(150, 150),
                                                  batch_size=20,
                                                  class_mode='binary')
test_loss, test_acc = model.evaluate_generator(test_generator, steps=50)
print('test acc:', test_acc)

In [ ]:
conv_base.trainable = True
set_trainable = False

for layer in conv_base.layers:
    if layer.name == 'block5_conv1':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False


In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-5),
              metrics=['acc'])

history = model.fit_generator(train_generator,
                              steps_per_epoch=100,
                              epochs=100,
                              validation_data=validation_generator,
                              validation_steps=50)

In [ ]:
# Displaying curves of loss and accuracy during training of model_5
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epoch = range(1, len(acc) + 1)

plt.plot(epoch, acc, 'bo', label='Training accuracy')
plt.plot(epoch, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')

plt.legend()
plt.show()

plt.plot(epoch, loss, 'ro', label='Training loss')
plt.plot(epoch, val_loss, 'r', label='Validation loss')
plt.title('Trainign and validation loss')

plt.legend()
plt.show()

In [ ]:
test_generator = test_datagen.flow_from_directory(
test_dir,
target_size=(150, 150),
batch_size=20,
class_mode='binary')
test_loss, test_acc = model.evaluate_generator(test_generator, steps=50)
print('test acc:', test_acc)

In [ ]:
testing_dir = "../input/dogs-vs-cats/test1"


In [ ]:
testing_generator = test_datagen.flow_from_directory(
    testing_dir,
    target_size=(150, 150),
    batch_size=1,
    shuffle = False,
    class_mode=None)

filenames = testing_generator.filenames
nb_samples = len(filenames)

predict = model.predict_generator(testing_generator)
predict_class = [1 if pred > 0.5 else 0 for pred in predict]
print(len(predict_class))

In [ ]:
#submission = pd.read_csv('../input/dogs-vs-cats/sampleSubmission.csv')
#submission.to_csv('submission.csv', index=False)
'''
out = open('../input/dogs-vs-cats/sampleSubmission.csv', "w")
out.write("id,label\n")
rows = ['']*preds.shape[0]
for num in range(1,len(predict_class)+1):
    rows[num]='%d,%d\n' % (num,preds[num-1])
out.writelines(rows)
out.close()
'''
ids = range(1, 12501)
my_submission = pd.DataFrame({'id': ids, 'label': predict_class})
# you could use any filename. We choose submission here
my_submission.to_csv('../working/submission.csv', index=False)